In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, time, random, cv2
from pathlib import Path
from PIL import Image
import imgaug as ia
from imgaug import augmenters as iaa
from tqdm import tqdm

from keras.models import Model
from keras.layers import (Conv2D, Input, Dense, Flatten, Dropout, GlobalAveragePooling2D,
                          Activation, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU)
from keras.optimizers import Adam, SGD, RMSprop
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [5]:
%matplotlib inline
pd.set_option('max_colwidth', 400)
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16
t_start = time.time()

DATA_DIR = "D:/dataset/Labelled/Labelled/"
CATEGORIES = os.listdir(DATA_DIR)
Ncategories = len(CATEGORIES)

In [6]:
# ----------------#
# Hyperparameters #
# ----------------#
Target_shape = (224, 224)
Test_size = 0.05
Nepochs = 10
Batch_size = 32
Learning_rate = 1e-04

In [8]:
data = []
for category_id, category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(DATA_DIR, category)):
        if file == ".directory": continue
        data.append(['{}{}/{}'.format(DATA_DIR, category, file), category_id, category])
data = pd.DataFrame(data, columns=['img_path', 'category_id', 'category'])
data['category_cat'] = list(to_categorical(data["category_id"], Ncategories))

data.sample(n=10, random_state=100).head(n=4)

,img_path,category_id,category,category_cat
2187,D:/dataset/Labelled/Labelled/Hispa/IMG_20190419_140353.jpg,2,Hispa,"[0.0, 0.0, 1.0, 0.0]"
2709,D:/dataset/Labelled/Labelled/LeafBlast/IMG_20190419_112433.jpg,3,LeafBlast,"[0.0, 0.0, 0.0, 1.0]"
1830,D:/dataset/Labelled/Labelled/Healthy/IMG_20190424_132239.jpg,1,Healthy,"[0.0, 1.0, 0.0, 0.0]"
452,D:/dataset/Labelled/Labelled/BrownSpot/IMG_20190424_134125.jpg,0,BrownSpot,"[1.0, 0.0, 0.0, 0.0]"


In [9]:
train, val = train_test_split(data, stratify=data.category, test_size=Test_size)
train, test = train_test_split(train, stratify=train.category, test_size=Test_size)
print("Training images: {}\nValidating images: {}\nTesting images: {}".format(len(train), len(val), len(test)))

Training images: 3027
Validating images: 168
Testing images: 160


In [10]:
# https://imgaug.readthedocs.io/en/latest/source/examples_keypoints.html
# https://github.com/aleju/imgaug
# https://imgaug.readthedocs.io/en/latest/source/augmenters.html
seq = iaa.Sequential([
    iaa.OneOf([
        iaa.Fliplr(0.5), # horizontally flip
        iaa.Flipud(0.5),
        iaa.Affine(rotate=45)
    ]),
    iaa.SomeOf(2, [
        iaa.AdditiveGaussianNoise(scale=0.2*255),
        iaa.Add(50, per_channel=True),
        iaa.Sharpen(alpha=0.2),
        iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)),
        iaa.Noop(),
        iaa.Dropout(p=(0, 0.2), per_channel=0.5),
        iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
    ])
    # More as you want ...
])
# seq_det = seq.to_deterministic()

In [11]:
def preprocess_input(img):
    #--- Rescale Image --- Rotate Image --- Resize Image --- Flip Image --- PCA etc.
    image = cv2.resize(img, Target_shape, interpolation = cv2.INTER_AREA)
    image = image/255.
    return(image)

def get_bach_balenced_idx(df, batch_size, nCategories):
    k = batch_size // nCategories
    idx = []
    for i in range(nCategories):
        _ = df[df['category_id']==i].sample(n=k).index.to_list()
        idx.extend(_)
    return idx
    
def batch_generator(df, batch_size = 16):
    while True:
        # Select files (paths/indices) for the batch
        df.reset_index(drop=True, inplace=True)
        idx = get_bach_balenced_idx(df, batch_size, 4)
        
        batch_paths = df.iloc[list(idx), :]["img_path"].values
        batch_outputs = df.iloc[list(idx), :]["category_cat"].values
        
        batch_input = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        for input_path, output_ in zip(batch_paths, batch_outputs):
            img = cv2.imread(str(input_path))
            input_ = preprocess_input(img)
            batch_input += [ input_ ]
            batch_output += [ output_ ]
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array( batch_input )
        # Augmentation for batch of images
        batch_x = seq.augment_images(batch_x)
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )

In [12]:
# https://keras.io/applications/
from keras.applications import (Xception, VGG16, VGG19, ResNet50, InceptionV3, 
                                InceptionResNetV2, MobileNet, DenseNet121, NASNetMobile)
def build_model(ModelName, InputShape = Target_shape):
    inp = Input((InputShape[0],InputShape[1],3))
    if ModelName == "Xception":
        base_model = Xception(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "VGG16":
        base_model = VGG16(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "VGG19":
        base_model = VGG19(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "ResNet50":
        base_model = ResNet50(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "InceptionV3":
        base_model = InceptionV3(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "InceptionResNetV2":
        base_model = InceptionResNetV2(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "MobileNet":
        base_model = MobileNet(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "DenseNet121":
        base_model = DenseNet121(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "NASNetMobile":
        base_model = NASNetMobile(input_tensor = inp, include_top=False, weights='imagenet')
        
    # frozen the first .8% layers
    NtrainableLayers = round(len(base_model.layers)*0.8)
    for layer in base_model.layers[:NtrainableLayers]:
        layer.trainable = False
    for layer in base_model.layers[NtrainableLayers:]:
        layer.trainable = True
    
    x_model = base_model.output
    x_model = GlobalAveragePooling2D(name='globalaveragepooling2d')(x_model)
    # x_model = Dense(1024, activation='relu',name='fc1_Dense')(x_model)
    # x_model = Dropout(0.5, name='dropout_1')(x_model)
    # x_model = Dense(256, activation='relu',name='fc2_Dense')(x_model)
    x_model = Dropout(0.1, name='dropout_2')(x_model)

    predictions = Dense(Ncategories, activation='softmax',name='output_layer')(x_model)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [14]:
# Callbacks for training: Make sure the training works well and doesnt run too long or overfit too much
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

MName = ("Xception")
BestModelWeightsPath = "{}_weights.best.hdf5".format(MName)

checkpoint = ModelCheckpoint(
    BestModelWeightsPath, 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max', 
    save_weights_only = True
)
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.2, 
    patience=3, 
    verbose=1, 
    mode='max', 
    cooldown=2, 
    min_lr=1e-7
)
early = EarlyStopping(
    monitor="val_accuracy", 
    mode="max", 
    patience=4
)

print(f"\n Transfer Learning with: {MName}: \n")

model = build_model(MName, InputShape = Target_shape)

model.compile(optimizer=Adam(lr=Learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

train_gen = batch_generator(train, Batch_size)
val_gen = batch_generator(val, Batch_size)

History = model.fit_generator(
    generator= train_gen,
    steps_per_epoch=train.shape[0]//Batch_size,
    validation_data=val_gen,
    validation_steps=val.shape[0]//Batch_size,
    epochs = Nepochs,
    callbacks = [checkpoint, early, reduceLROnPlat]
)


 Transfer Learning with: Xception: 


Epoch 1/10


ValueError: Got dtype 'float64', which is a forbidden dtype (uint32, uint64, uint128, uint256, int32, int64, int128, int256, float96, float64, float128, float256).